In [1]:
#分箱操作
import pandas as pd
data = pd.read_csv('data_train.csv',index_col=0,header=0)

    https://blog.csdn.net/kxiaozhuk/article/details/84612632
    

In [2]:
#####代码中需要引用的包#####
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE, f_regression
import scipy.stats.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import math
import matplotlib.pyplot as plt


In [3]:
'''#提取样本训练集和测试集
def split_data(data, ratio=0.7, seed=None):
    if seed:
        shuffle_data = shuffle(data, random_state=seed)
    else:
        shuffle_data = shuffle(data, random_state=np.random.randint(10000))
    train = shuffle_data.iloc[:int(ratio*len(shuffle_data)), ]
    test = shuffle_data.iloc[int(ratio*len(shuffle_data)):, ]
    return train, test
 
#设置seed是为了保证下次拆分的结果一致
df_train,df_test = split_data(data, ratio=0.7, seed=666)
#将违约样本用“1”表示，正常样本用“0”表示。
credit_risk = [0 if x=='good' else 1 for x in df_train['y']]
#credit_risk = np.where(df_train['credit_risk'] == 'good',0,1)
data = df_train
'''

"#提取样本训练集和测试集\ndef split_data(data, ratio=0.7, seed=None):\n    if seed:\n        shuffle_data = shuffle(data, random_state=seed)\n    else:\n        shuffle_data = shuffle(data, random_state=np.random.randint(10000))\n    train = shuffle_data.iloc[:int(ratio*len(shuffle_data)), ]\n    test = shuffle_data.iloc[int(ratio*len(shuffle_data)):, ]\n    return train, test\n \n#设置seed是为了保证下次拆分的结果一致\ndf_train,df_test = split_data(data, ratio=0.7, seed=666)\n#将违约样本用“1”表示，正常样本用“0”表示。\ncredit_risk = [0 if x=='good' else 1 for x in df_train['y']]\n#credit_risk = np.where(df_train['credit_risk'] == 'good',0,1)\ndata = df_train\n"

In [4]:
data.shape

(7500, 30)

In [5]:
lisan = [ 'mar_status_0.0', 'mar_status_1.0',
       'mar_status_2.0']

In [6]:
lianxu=['y', 'age', 'xaccount_age', 'cred_limit', 'this_bill_rate',
       'this_bill_mp', 'last_mp_days', 'six_bill_num', 'six_bill_avg_amt',
       'six_cycle_mp_num', 'six_cycle_mp_avg_amt', 'six_bill_avg_debt_rate',
       'six_bill_low_repay_num', 'epp_nbr_12m', 'alop_nbr_12m',
       'epp_avg_amt_12m', 'alop_avg_amt_12m', 'consume_num_session12',
       'consume_num_session6', 'consume_num_session3', 'consume_num_session',
       'consume_amt_session12', 'consume_amt_session6', 'consume_amt_session3',
       'consume_amt_session', 'month_avg_use_year', 'month_avg_use_month6',
       'month_avg_use_month3']

In [7]:
data.columns.values

array(['city', 'mar_status', 'age', 'xaccount_age', 'cred_limit',
       'this_bill_rate', 'this_bill_mp', 'last_mp_days', 'six_bill_num',
       'six_bill_avg_amt', 'six_cycle_mp_num', 'six_cycle_mp_avg_amt',
       'six_bill_avg_debt_rate', 'six_bill_low_repay_num', 'epp_nbr_12m',
       'alop_nbr_12m', 'epp_avg_amt_12m', 'alop_avg_amt_12m',
       'consume_num_session12', 'consume_num_session6',
       'consume_num_session3', 'consume_num_session',
       'consume_amt_session12', 'consume_amt_session6',
       'consume_amt_session3', 'consume_amt_session',
       'month_avg_use_year', 'month_avg_use_month6',
       'month_avg_use_month3', 'y'], dtype=object)

In [8]:
df_feature = pd.DataFrame(data,columns=[ 'age', 'xaccount_age', 'cred_limit', 'this_bill_rate',
       'this_bill_mp', 'last_mp_days', 'six_bill_num', 'six_bill_avg_amt',
       'six_cycle_mp_num', 'six_cycle_mp_avg_amt',
       'six_bill_avg_debt_rate', 'six_bill_low_repay_num', 'epp_nbr_12m',
       'alop_nbr_12m', 'epp_avg_amt_12m', 'alop_avg_amt_12m',
       'consume_num_session12', 'consume_num_session6',
       'consume_num_session3', 'consume_num_session',
       'consume_amt_session12', 'consume_amt_session6',
       'consume_amt_session3', 'consume_amt_session', 'month_avg_use_year',
       'month_avg_use_month6', 'month_avg_use_month3'])
df_feature.head()
#f_regression(df_feature,credit_risk)
#, 'mar_status_0.0',       'mar_status_1.0', 'mar_status_2.0'

,age,xaccount_age,cred_limit,this_bill_rate,this_bill_mp,last_mp_days,six_bill_num,six_bill_avg_amt,six_cycle_mp_num,six_cycle_mp_avg_amt,...,consume_num_session6,consume_num_session3,consume_num_session,consume_amt_session12,consume_amt_session6,consume_amt_session3,consume_amt_session,month_avg_use_year,month_avg_use_month6,month_avg_use_month3
2967,45.0,62.0,17000.0,0.044703,721.95,9999.0,6.0,1594.75,0.0,0.0,...,52.0,34.0,15.0,18469.71,13565.39,3469.92,927.96,0.090538,0.132994,0.068038
700,24.0,15.0,28000.0,0.765756,20387.01,43.0,6.0,14122.09,1.0,4700.0,...,195.0,83.0,21.0,123695.97,99878.75,72008.67,22016.18,0.454765,0.657097,0.857246
3481,40.0,55.0,19500.0,0.979458,17834.13,956.0,6.0,14701.03,0.0,0.0,...,24.0,15.0,6.0,143169.00,89433.00,56497.00,19090.00,0.611833,0.764385,0.965761
1621,34.0,15.0,24000.0,0.087500,1995.00,9999.0,6.0,7423.88,0.0,0.0,...,2.0,2.0,1.0,13600.00,13600.00,13600.00,2100.00,0.283333,0.283333,0.283333
800,48.0,21.0,15000.0,0.931308,12868.35,9999.0,6.0,9349.12,0.0,0.0,...,20.0,12.0,7.0,55154.63,40202.63,26205.63,13533.63,0.306415,0.446696,0.582347


In [12]:
df_feature = data.drop(['y','mar_status'],axis=1)

In [13]:

f_regression( data.drop(['y','mar_status'],axis=1),data['y']) #第二列是p值 输出逐步回归后得到的变量，选择P值<=0.1的变量

(array([4.24944966e+00, 7.40867159e+00, 6.69864908e+00, 1.83917392e+01,
        1.62824073e+01, 2.00765057e+01, 7.99781769e+02, 4.46157148e+00,
        7.69152023e+00, 1.82866207e+03, 1.50057255e+02, 1.98030632e+00,
        7.31518063e+01, 1.01439870e+03, 1.04705016e+02, 3.89577326e+02,
        3.27314802e+01, 3.04584288e+00, 9.13285646e-01, 5.94742321e-01,
        5.77928381e-01, 1.22479764e+00, 4.20981510e+00, 6.13602791e+00,
        3.05746682e+01, 5.79863143e+00, 4.60315757e+00, 6.60051331e+00]),
 array([3.92974472e-002, 6.50601986e-003, 9.66721642e-003, 1.82091031e-005,
        5.51105588e-005, 7.55091008e-006, 2.84299447e-167, 3.46985698e-002,
        5.56179933e-003, 0.00000000e+000, 3.56865655e-034, 1.59399590e-001,
        1.44029032e-017, 6.78581996e-209, 2.05013877e-024, 1.39730716e-084,
        1.09882357e-008, 8.09850510e-002, 3.39274950e-001, 4.40615431e-001,
        4.47149380e-001, 2.68456669e-001, 4.02253733e-002, 1.32673323e-002,
        3.32070239e-008, 1.60626406e-0